# Our device predicts the probability of an individual facing sexual harassment/crime.

### Project by PAASS.

### Instructions to move character around:
1. Move Thumbstick to Direction, note you are controlling blue.
2. Choose whether you wanna move or look, do so by clicking the option of either button.
3. Watch the probability on the the top right increase or decrease according to your movements.

Importing all required modules

In [37]:
import tkinter as tk
import math
import serial
import time
import random
from scipy.stats import linregress
import numpy as np
from tkinter import *

Making a character to be used on the grid

In [38]:
class Character:
    def __init__(self, x, y, direction,color=""):
        self.color=color
        self.x = x
        self.y = y
        self.direction = direction
    
    def look(self, rangey=30):
        # Initialize an empty list to store the coordinates within range squares in front of the character
        coordinates = []
        
        # Iterate over the range of distances
        for distance in range(1, rangey + 1):
            # Add the coordinates distance squares in front of the character to the list
            if self.direction == "up":
                coordinates.append((self.x, self.y - distance))
            elif self.direction == "down":
                coordinates.append((self.x, self.y + distance))
            elif self.direction == "left":
                coordinates.append((self.x - distance, self.y))
            elif self.direction == "right":
                coordinates.append((self.x + distance, self.y))
            elif self.direction == "up-left":
                coordinates.append((self.x - distance, self.y - distance))
            elif self.direction == "up-right":
                coordinates.append((self.x + distance, self.y - distance))
            elif self.direction == "down-left":
                coordinates.append((self.x - distance, self.y + distance))
            elif self.direction == "down-right":
                coordinates.append((self.x + distance, self.y + distance))
        
        # Return the list of coordinates
        return coordinates

    
    def move(self):
        # Update the character's position based on their current direction
        if self.direction == "up":
            self.y -= 1
        elif self.direction == "down":
            self.y += 1
        elif self.direction == "left":
            self.x -= 1
        elif self.direction == "right":
            self.x += 1
        elif self.direction == "up-left":
            self.x -= 1
            self.y -= 1
        elif self.direction == "up-right":
            self.x += 1
            self.y -= 1
        elif self.direction == "down-left":
            self.x -= 1
            self.y += 1
        elif self.direction == "down-right":
            self.x += 1
            self.y += 1
    
    def distance_to(self, other_character):
        # Calculate the distance between this character and another character
        x_distance = other_character.x - self.x
        y_distance = other_character.y - self.y
        distance = math.sqrt(x_distance ** 2 + y_distance ** 2)
        return distance
    
    def is_looking_at(self, other_character):
        # Check if this character is looking at another character
        coordinates_in_front = self.look()
        for i in coordinates_in_front:
            if i == (other_character.x, other_character.y):
                return True
        return False

### Plotting characters on grid using tkinter

In [39]:
def plot_characters(characters, grid_size, ypredfinal=""):
    # Initialize the main window
    window = tk.Tk()
    window.title("Character Grid")

    # Create a canvas to draw on
    canvas = tk.Canvas(window, width=grid_size*50, height=grid_size*50)
    canvas.pack()

    # Draw a grid of squares on the canvas
    for i in range(grid_size):
        for j in range(grid_size):
            # Check if the square is in the direction of any character
            looking_at = False
            for character in characters:
                if (i, j) in character.look():
                    looking_at = True
                    break
            # Fill the square with red color if it is in the direction of a character, otherwise fill it with white
            if looking_at:
                canvas.create_rectangle(i*50, j*50, (i+1)*50, (j+1)*50, fill="red")
            else:
                canvas.create_rectangle(i*50, j*50, (i+1)*50, (j+1)*50, fill="white")
    # Draw a circle for each character on the canvas
    for character in characters:
        x = character.x
        y = character.y
        # Check if this character is looking at any other character
        looking_at = False
        for other_character in characters:
            if character is not other_character and character.is_looking_at(other_character):
                looking_at = True
                break
        # Fill the circle with orange color if the character is looking at another character, otherwise fill it with red
        if looking_at:
            canvas.create_oval(x*50+10, y*50+10, x*50+40, y*50+40, fill="orange")
        else:
            canvas.create_oval(x*50+10, y*50+10, x*50+40, y*50+40, fill=character.color)

    # Display the value of ypredfinal on the top right corner of the grid
    canvas.create_text(grid_size*50 - 10, 10, anchor="ne", text=str(ypredfinal))

    # Schedule the window to be destroyed after 3 seconds
    window.after(3000, window.destroy)

    # Run the tkinter event loop
    window.mainloop()

### Computing the probability of sexual harassment using regression.

In [40]:
def probabilitytrue(distance):
    distancesX=np.array([i for i in range(1,30)])
    probabilitiesiftrue=np.array([probability(k,True) for k in distancesX])
    slope, intercept, r_value, p_value, std_err = linregress(distancesX, probabilitiesiftrue)
    y_pred = slope * distance + intercept
    return y_pred

def probabilityfalse(distance):
    distancesX=np.array([i for i in range(1,30)])
    probabilitiesiffalse=np.array([probability(k,False) for k in distancesX])
    slope, intercept, r_value, p_value, std_err = linregress(distancesX, probabilitiesiffalse)
    y_pred = slope * distance + intercept
    return y_pred

def whichy(is_look,distance):
    if is_look==True:
        return probabilitytrue(distance)
    if is_look==False:
        return probabilityfalse(distance)
        
def probability(distance,is_look=False):
    probability=0
    if distance>=1 and distance<=3 and is_look==True:
        probability=90
    if distance>3 and distance<=5 and is_look==True:
        probability=70
    if distance>5 and distance<=10 and is_look==True:
        probability=50
    if distance>10 and is_look==True:
        probability= 50-distance
    if distance>=1 and distance<=3 and is_look==False:
        probability=50
    if distance>3 and distance<=5 and is_look==False:
        probability=35
    if distance>5 and distance<=10 and is_look==False:
        probability=15
    return probability

# GUI to move/look around

In [41]:
def get_modeinput():
    def get_move_mode():
        global mode
        mode = "move"
        root.destroy()  # close the GUI window

    def get_look_mode():
        global mode
        mode = "look"
        root.destroy()  # close the GUI window

    # create the main window
    root = Tk()

    # create two buttons to trigger the corresponding functions
    move_button = Button(root, text="Move", command=get_move_mode)
    move_button.pack()
    look_button = Button(root, text="Look", command=get_look_mode)
    look_button.pack()

    # start the main loop
    root.mainloop()

Testing to see if it works/demo for simulation purposes

In [42]:
girl=Character(random.randint(0,8),random.randint(0,8),"up","pink")
boy=Character(random.randint(0,8),random.randint(0,8),"up","blue")
# Open the serial port
ser = serial.Serial("COM6", 9600)
while True:
    # Read a line from the serial port
    line = ser.readline().decode("utf-8").strip()
    # Print the direction received from the Arduino
    newline=""
    while line!="Center":
        newline=line
        print(line)
        is_look=boy.is_looking_at(girl)
        plot_characters([boy,girl],9)        
        get_modeinput()
        time.sleep(1)
        line = ser.readline().decode("utf-8").strip()
    if newline!="" and mode=="look":
        boy.direction=newline
        is_look=boy.is_looking_at(girl)
        ypredfinal=str(round(whichy(is_look,boy.distance_to(girl)),2))+"%"
        plot_characters([boy,girl],9,ypredfinal)
    if newline!="" and mode=="move":
        boy.direction=newline
        is_look=boy.is_looking_at(girl)
        boy.move()
        ypredfinal=str(round(whichy(is_look,boy.distance_to(girl)),2))+"%"
        plot_characters([boy,girl],9,ypredfinal)

right
up
up
up
up
right


KeyboardInterrupt: 